In [3]:
import wrds
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import datetime
from scipy import stats
import pickle
from pandas.tseries.offsets import BDay
from pandas.tseries.offsets import DateOffset
T=30

In [4]:
spx_front = pd.read_csv('data\\spx_front.csv').iloc[:,2:]
spx_next = pd.read_csv('data\\spx_next.csv').iloc[:,2:]
spx_spot = pd.read_csv('data\\spx_stock.csv',index_col=0,header=None)

spx_front = spx_front[~spx_front.duplicated()]
spx_next = spx_next[~spx_next.duplicated()]

spx_front.date = pd.to_datetime(spx_front.date)
spx_front.exdate = pd.to_datetime(spx_front.exdate)

spx_next.date = pd.to_datetime(spx_next.date)
spx_next.exdate = pd.to_datetime(spx_next.exdate)

spx_spot.index = pd.to_datetime(spx_spot.index)
spx_spot.columns = ["close"]

spx_dates = pd.to_datetime(sorted(spx_front.date.unique()))
spx_dates = spx_dates[spx_dates>=pd.to_datetime('1996-07-01')]

In [2]:
def get_otm(optiondf,secdf):
    
    optiondf['option_price']=optiondf['mid_price']
    optiondf['tao']=(optiondf['exdate']-optiondf['date']).apply(lambda x:x.days)

    #1.join spot and option price
    df1=optiondf.join(secdf,on='date')
    #optiondf has no index and secdf has index of date
    
    #2.join Call and put on the same day of same strike
    df1=df1.set_index(['date','strike_price'])

    calldf=df1[df1['cp_flag']=='C']
    putdf=df1[df1['cp_flag']=='P']
    df2=calldf.join(putdf['option_price'],how='inner',lsuffix='_call',rsuffix='_put')

    #3.select K which has the minimum absolute difference between call and put
    #and calculate F 
    df2['absdiff']=abs(df2['option_price_call']-df2['option_price_put'])
    df2['diff']=df2['option_price_call']-df2['option_price_put']
    df3=df2.loc[df2.groupby(['date'])['absdiff'].idxmin()]

    df3=df3.reset_index()
    df3['F']=df3['strike_price']+df3['forward_price']/df3['close']*df3['diff']

    #4.calculate K0
    df2=df2.reset_index()
    df3=df3.set_index(['date'])
    df4=df2.join(df3['F'],on='date',how='left')
    df5=df4[df4['F']-df4['strike_price']>0]
    df5['Fkdiff']=df5['F']-df5['strike_price']
    df6=df5.loc[df5.groupby(['date'])['Fkdiff'].idxmin()]
    df6['K0']=df6['strike_price']


    df6=df6.set_index(['date'])

    calldf=calldf.reset_index()
    calldf=calldf.set_index('date')
    calldff=calldf.join(df6['K0'],on='date',how='left')
    callotm=calldff[calldff['strike_price']>calldff['K0']]
    callitm=calldff[calldff['strike_price']<calldff['K0']]
    

    putdf=putdf.reset_index()
    putdf=putdf.set_index('date')
    putdff=putdf.join(df6['K0'],on='date',how='left')
    putotm=putdff[putdff['strike_price']<putdff['K0']]
    putitm=putdff[putdff['strike_price']>putdff['K0']]


    #df6:atm
    df6['option_price']=0.5*(df6['option_price_call']+df6['option_price_put'])
    df6=df6[callotm.columns]

    
    return callotm,putotm,callitm,putitm,df6

In [9]:
callotmf,putotmf,callitmf,putitmf,atmf = get_otm(spx_front,spx_spot)
callotmn,putotmn,callitmn,putitmn,atmn = get_otm(spx_next,spx_spot)

In [10]:
allf=pd.concat([callotmf,putotmf,atmf],axis=0)
alln=pd.concat([callotmn,putotmn,atmn],axis=0)

In [11]:
itmf=pd.concat([callitmf,putitmf],axis=0)##index:date
itmn=pd.concat([callitmn,putitmn],axis=0)

In [12]:
def createFeatures(space,num,atmdf,alldf):
    
    atmdf=atmdf[['exdate','strike_price','option_price']]
    df=pd.DataFrame()
    for d in range(len(atmdf)):
        d1=pd.DataFrame([atmdf.iloc[d,:]]*(2*num+1))
        d1['strike_price']=np.linspace(atm_n['strike_price'].iloc[d]-num*space,atm_n['strike_price'].iloc[d]+num*space,2*num+1)
        df=pd.concat([df,d1],axis=0)
    
    df=df.reset_index()

    alldf=alldf.reset_index()
    alldf=alldf.set_index(['date','strike_price'])
    
    featuredf=df.join(alldf['option_price'],on=['index','strike_price'],lsuffix='_temp')

    for s in len(featuredf):
        if fea
    #featuredf.drop(columns='option_price_temp',inplace=True)

    return featuredf

SyntaxError: invalid syntax (<ipython-input-12-642ec7ea922f>, line 18)

In [8]:
featuredf=createFeatures(5,5,atmn,allf)

NameError: name 'createFeatures' is not defined

# Call Put Front Next

In [75]:
def get_idate_opt(callotmf,idate):
    ioptcf = callotmf[callotmf.index == idate]

    ioptcf.sort_values(by="strike_price",inplace = True)

    ioptcf.set_index("strike_price",inplace=True)
    return ioptcf

In [104]:
def get_features(callotmf,callotmn,putotmf,putotmn,n):
    feature_list = []

    for i in range(len(spx_dates)):
        idate = spx_dates[i]
        print(idate)
        ioptcf = get_idate_opt(callotmf,idate)
        ioptcn = get_idate_opt(callotmn,idate)

        ioptpf = get_idate_opt(putotmf,idate)
        ioptpn = get_idate_opt(putotmn,idate)

        concat_call = ioptcf.join(ioptcn,how='inner',lsuffix='_f',rsuffix='_n')

        concat_call['weighted_price'] = ((concat_call["tao_n"]-T)*concat_call["mid_price_f"]+(T-concat_call["tao_f"])*concat_call["mid_price_n"])/(concat_call["tao_n"]-concat_call["tao_f"])

        concat_put = ioptpf.join(ioptpn,how='inner',lsuffix='_f',rsuffix='_n')

        concat_put['weighted_price'] = (concat_put["tao_n"]-T)*concat_put["mid_price_f"]+((T-concat_put["tao_f"])*concat_put["mid_price_n"])/(concat_put["tao_n"] - concat_put["tao_f"])

        concat_all = pd.concat([concat_put,concat_call],axis=0)
        concat_all["abs_diff"] = abs(concat_all.index.values - concat_all.K0_f.values)
        try:
            min_iloc = concat_all.index.get_loc(concat_all.abs_diff.idxmin())
            features = concat_all.iloc[min_iloc-n:min_iloc+n+1,:]["weighted_price"].values
        except:
            features = [np.nan]*(2*n+1)
            
        feature_list.append(features)
        feature_df = pd.DataFrame(feature_list)

    return feature_df

In [105]:
feature_df = get_features(callotmf,callotmn,putotmf,putotmn,5)

2015-07-10 00:00:00
2015-07-13 00:00:00
2015-07-14 00:00:00
2015-07-15 00:00:00
2015-07-16 00:00:00
2015-07-17 00:00:00
2015-07-20 00:00:00
2015-07-21 00:00:00
2015-07-22 00:00:00
2015-07-23 00:00:00
2015-07-24 00:00:00
2015-07-27 00:00:00
2015-07-28 00:00:00
2015-07-29 00:00:00
2015-07-30 00:00:00
2015-07-31 00:00:00
2015-08-03 00:00:00
2015-08-04 00:00:00
2015-08-05 00:00:00
2015-08-06 00:00:00
2015-08-07 00:00:00
2015-08-10 00:00:00
2015-08-11 00:00:00
2015-08-12 00:00:00
2015-08-13 00:00:00
2015-08-14 00:00:00
2015-08-17 00:00:00
2015-08-18 00:00:00
2015-08-19 00:00:00
2015-08-20 00:00:00
2015-08-21 00:00:00
2015-08-24 00:00:00
2015-08-25 00:00:00
2015-08-26 00:00:00
2015-08-27 00:00:00
2015-08-28 00:00:00
2015-08-31 00:00:00
2015-09-01 00:00:00
2015-09-02 00:00:00
2015-09-03 00:00:00
2015-09-04 00:00:00
2015-09-08 00:00:00
2015-09-09 00:00:00
2015-09-10 00:00:00
2015-09-11 00:00:00
2015-09-14 00:00:00
2015-09-15 00:00:00
2015-09-16 00:00:00
2015-09-17 00:00:00
2015-09-18 00:00:00


In [137]:
feature_df.index  = spx_dates

In [139]:
feature_df.to_csv("features_11.csv")

## Vol target

In [14]:
spx_return = spx_spot.pct_change()+1

spx_log_return = np.log(spx_return)
spx_var = spx_log_return.copy()
spx_var.iloc[:,0] =  np.nan

for i in range(len(spx_log_return)-30):
    var = np.var(spx_log_return.iloc[i:i+30,0])
    date = spx_log_return.index[i]
    spx_var.iloc[i,0] = var

spx_spot["log_return"] = spx_log_return.values
spx_spot["variance_30"] = spx_var.values
spx_spot["vol_30"] = np.sqrt(spx_var.values)
spx_spot["ann_vol_30"] = np.sqrt(spx_var.values) * np.sqrt(252) *100
spx_spot.to_csv("data/spx_vol.csv")

## vix

In [15]:
vix = pd.read_csv("data/vix.csv",index_col=0)
vix.index = pd.to_datetime(vix.index)
spx_spot= spx_spot.join(vix["Close"],how="inner",rsuffix="_vix")
spx_spot["vix"] = spx_spot["Close"]
spx_target = spx_spot.loc[:,["ann_vol_30","vix"]]

spx_target.to_csv("data/spx_target.csv")

In [17]:
spx_target

,ann_vol_30,vix
1996-07-01,16.109703,13.780000
1996-07-02,15.997377,14.190000
1996-07-03,16.010210,14.210000
1996-07-05,16.008138,16.090000
1996-07-08,14.684351,16.420000
1996-07-09,14.505247,15.490000
1996-07-10,14.494548,15.510000
1996-07-11,14.494039,17.809999
1996-07-12,13.822966,17.090000
1996-07-15,13.955084,20.110001
